In [1]:
import pandas as pd
import numpy as np
import subprocess
import re
from subprocess import Popen, PIPE

In [2]:
sample_sub = pd.read_csv('sample_submission.csv')
sample_sub

,id,EAP,HPL,MWS
0,id02310,0.403494,0.287808,0.308698
1,id24541,0.403494,0.287808,0.308698
2,id00134,0.403494,0.287808,0.308698
3,id27757,0.403494,0.287808,0.308698
4,id04081,0.403494,0.287808,0.308698
5,id27337,0.403494,0.287808,0.308698
6,id24265,0.403494,0.287808,0.308698
7,id25917,0.403494,0.287808,0.308698
8,id04951,0.403494,0.287808,0.308698
9,id14549,0.403494,0.287808,0.308698


In [3]:
def write_content_txt_file(content):
    clean_txt_file()
    with open('test.txt', 'w') as f:
        f.write(content)
    return None

def clean_txt_file():
    f = open('test.txt', 'r+')
    f.truncate()
    return None

## Data cleaning functions
def remove_label(text):
    txt = re.sub('__label__', '', text)
    return txt

def remove_new_line(text):
    txt = re.sub('\n', '', text)
    return txt

In [4]:
test = pd.read_csv('test.csv')
test

,id,text
0,id02310,"Still, as I urged our leaving Ireland with suc..."
1,id24541,"If a fire wanted fanning, it could readily be ..."
2,id00134,And when they had broken down the frail door t...
3,id27757,While I was thinking how I should possibly man...
4,id04081,I am not sure to what limit his knowledge may ...
5,id27337,"""The thick and peculiar mist, or smoke, which ..."
6,id24265,"That which is not matter, is not at all unless..."
7,id25917,I sought for repose although I did not hope fo...
8,id04951,"Upon the fourth day of the assassination, a pa..."
9,id14549,"""The tone metaphysical is also a good one."


In [5]:
with(open('test_frases.txt', 'w')) as f:
    for i in range(len(test)):
        f.write(test['text'][i].lower()+"\n")

In [6]:
my_sub = pd.DataFrame(columns={'id', 'EAP','HPL', 'MWS'})
my_sub=my_sub[['id', 'EAP','HPL', 'MWS']]
my_sub

,id,EAP,HPL,MWS


In [7]:
fast = Popen(['fasttext', 'predict-prob', 'model_1.bin','test_frases.txt', '3'], stdin=PIPE, stdout=PIPE, stderr=PIPE)
output, err = fast.communicate()
#print(output)
reply = "".join(output.decode('utf8')).split("\n")
for i in range(len(reply)):
    if re.search('__label__',reply[i]):
        reply[i]=remove_label(reply[i])
    if re.search('\n',reply[i]):
        reply[i]=remove_new_line(reply[i])

## Clearning empty replies
reply[:] = [item for item in reply if item !='']
outer =[]
for i in range(len(reply)):
    outer.append(reply[i].split(" "))
    
for i in range(len(outer)):
    for j in range(len(outer[i])):
        if re.search('EAP', outer[i][j]):
            EAP = outer[i][j+1]
        if re.search('MWS', outer[i][j]):
            MWS = outer[i][j+1]
        if re.search('HPL', outer[i][j]):
            HPL = outer[i][j+1]
    my_sub.loc[i] = [test['id'][i], EAP, HPL, MWS]

In [8]:
my_sub

,id,EAP,HPL,MWS
0,id02310,1.95313e-08,1.95313e-08,0.998047
1,id24541,1,1.95313e-08,1.95313e-08
2,id00134,1.95313e-08,0.998047,1.95313e-08
3,id27757,0.998047,1.95313e-08,1.95313e-08
4,id04081,0.402344,0.595703,1.95313e-08
5,id27337,1,1.95313e-08,1.95313e-08
6,id24265,1,1.95313e-08,1.95313e-08
7,id25917,1.95313e-08,1.95313e-08,0.998047
8,id04951,1,1.95313e-08,1.95313e-08
9,id14549,1,1.95313e-08,1.95313e-08


In [9]:
my_sub.to_csv('roberto_5.csv',index=False)